In [1]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [2]:
this_month='1709'
data_path='../data/'+this_month

In [3]:
df=pd.read_csv(data_path+'/cleaned.csv')
df.shape

(10867268, 15)

In [4]:
df=df[~((df.year==2014) & (df.mm<9))]
df.shape

(8818249, 15)

In [5]:
df=df[(df['year']<=2017) & (df['year']>=2014)]
df.shape

(8818249, 15)

In [6]:
unique_subjects=pd.read_csv('../store/unique_subjects.csv',header=None)
unique_subjects=unique_subjects[0].values

In [7]:
with open('../store/issn_subject.json') as data_file:    
    issn_subject = json.load(data_file)

In [8]:
df2=df[['doi','main_issn','citation']]

In [9]:
join_subjects=df2['main_issn'].apply(lambda x: ';'.join(issn_subject[x]) if x in issn_subject else '')

In [14]:
df2['subjects']=join_subjects.values

C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
df2['main_issn'].apply(lambda x: ';'.join(issn_subject[x]) if x in issn_subject else '').shape

(8818249L,)

In [16]:
unique_subjects=[]
all_subjects=df2['subjects'].values
for journal_subjects in all_subjects:
    subjects=journal_subjects.split(';')
    for subject in subjects:
        if subject not in unique_subjects: unique_subjects.append(subject)

In [17]:
len(unique_subjects)

327

In [ ]:
top_articles=[]
counter=0
for subject in unique_subjects:
    if subject=='':
        df_subject=df2.sort_values('citation', ascending= False)[:20]
    else:
        df_subject=df2[df2.subjects.apply(lambda x: subject in x.split(';'))].sort_values('citation', ascending= False)[:20]
    top_dois=df_subject[['doi','main_issn','citation']].to_dict('list')
    zips=zip(top_dois['doi'],top_dois['main_issn'],top_dois['citation'])
    items=[];
    for doi,issn,citation in zips:
        item={}; item_cleaned={}
        url='http://api.crossref.org/works/'+doi
        while True:
            try:
                response = urllib.urlopen(url).read()    
            except Exception,e: 
                print str(e)
                time.sleep(1)
                continue
            break
        json_result=json.loads(response)
        item = json_result['message']
        year_print=[9999,99,99];year_online=[9999,99,99];
        if 'title' in item:
            title=item['title'][0]
        if 'published-print' in item:
            year_print=item['published-print']['date-parts'][0]                
        if 'published-online' in item:
            year_online=item['published-online']['date-parts'][0]
        year=min(year_print,year_online)
        if 'DOI' in item: doi=item['DOI']
        if 'container-title' in item: journal=','.join(item['container-title'])
        if 'publisher' in item: pub=item['publisher']; pub=''if pub==None else pub
        if 'references-count' in item: ref=str(item['references-count'])
        authors=[]
        if 'author' in item: 
            for author in item['author']:
                if 'given' in author:
                    author_name=author['given']
                if 'family' in author:
                    author_name+=' '+author['family']
                authors.append(author_name)
        if len(authors)>5: authors=authors[:5]
        if len(year)>1 :
            month=str(year[1]) 
        else: month=''
        item_cleaned={'@id':doi,
                      'e:title':title,
                      'e:year':str(year[0])+'-'+month,
                      'e:cite':np.asscalar(citation),
                      'e:issn':issn, 
                      'e:journal':journal,
                      'e:pub':pub,
                      'e:authors':authors,
                      'e:subject':subject,
                      'e:month':this_month} 
        if item_cleaned not in top_articles:
            top_articles.append(item_cleaned)
    print counter,
    counter+=1            

0 1 2 3

In [ ]:
with open('../jsonld/'+this_month+'/top_articles.jsonld', 'w') as fp:
    json.dump(top_articles, fp)    

for subject in unique_subjects:
    df_lic=df2[df2.subjects.apply(lambda x: subject in x.split(';'))]
    top_lics=df_lic[['lic','count']].groupby('lic').count().sort_values('count',ascending=False).head().reset_index().to_dict('list')
    zips=zip(top_lics['lic'],top_lics['count'])
    total_articles=str(df_lic.shape[0])
    items=[{'count':total_articles,'lic':'all'}];
    for lic,count in zips:
        item={}; 
        item={'lic':lic,'count':str(count)}
        items.append(item)  
    with open('../jsonld/lic_'+subject+'.json', 'w') as fp:
        json.dump(items, fp)    